In [1]:
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
from util import *

In [2]:
pd.set_option('max_columns', None)
pd.set_option("max_rows", None)

In [3]:
hist_df = pd.read_parquet("data/trans_hist_1m.snappy.parquet")
hist_df.head()

,trans_amount,ben_cust_name,msisdn,process_code,error_code,correct_code,request_date
0,1000000.0,NGUYEN THI LIEN,0001caa88ddf200efd2c,641000,00,00,2020-08-14 12:33:05
1,300000.0,PHAM THI LAN HUONG,0001238e025a0f131d4e,640001,00,00,2020-08-14 09:09:30
2,400000.0,TRAN PHUONG THAO,0001a233794b9745400c,640000,00,00,2020-08-14 02:47:13
3,29940000.0,CONG TY TNHH SAO XANH,0001ceb9bebc5fbcdfb8,640001,00,00,2020-08-14 08:17:05
4,162864.0,None,0001aff517869188ab1e,300001,16,16,2020-08-14 02:32:02


In [4]:
hist_df.shape

(14733736, 7)

In [5]:
hist_df = hist_df.groupby(['msisdn', 'ben_cust_name'], as_index=False).agg({'trans_amount':['sum', 'mean']})
hist_df.head()

msisdn                              ben_cust_name  \
                                                                     
0  00010000021da335cd1c  HAN-LE-Vinhomes Times city Park Hill ca 2   
1  00010000029e7e24958a                                 VÀNG MÍ VỪ   
2  0001000012afca2aff7c                            DUONG XUAN LINH   
3  0001000012afca2aff7c                           NGUYEN TRAN THAO   
4  0001000012afca2aff7c                           Nguyen Tran Thao   

  trans_amount             
           sum       mean  
0   18033000.0  9016500.0  
1      50000.0    50000.0  
2    1200000.0  1200000.0  
3   11740000.0  2348000.0  
4     649106.0   649106.0

In [6]:
hist_df.columns = ['_'.join(col) for col in hist_df.columns]

In [7]:
hist_df.columns

Index(['msisdn_', 'ben_cust_name_', 'trans_amount_sum', 'trans_amount_mean'], dtype='object')

In [8]:
use_cols = [
    "province_code",
    "district_code",
    "viettel_bank_code",
    "channel_type_id",
    "channel_code",
    "staff_code",
    "trans_amount",
    "trans_fee",
    "trans_type",
    "process_code",
    "channel_fee",
    "shop_code",
    "customer_fee",
    "cust_name",
    "ben_cust_name",
    "msisdn",
    "ben_msisdn",
    "msisdn_channel",
    "staff_name"
]
train_df = pd.read_parquet("data/data-202008.snappy.parquet", columns=use_cols)
train_df.head()

,province_code,district_code,viettel_bank_code,channel_type_id,channel_code,staff_code,trans_amount,trans_fee,trans_type,process_code,channel_fee,shop_code,customer_fee,cust_name,ben_cust_name,msisdn,ben_msisdn,msisdn_channel,staff_name
0,D501,D501008,MB,164,HOANT35_VTPOST_DKN,HOANT35_VTPOST_DKN,500000.0,8182.0,1,645200,5400.0,1800170007,18000.0,HỒ THỊ KIM TRANG,MAI VAN QUY,0001b3807ce1879a06eb,0001b3807ce1879a06eb,000106f0196c5e4f0853,Nguyễn Thị Hoa
1,T008,T00807,MB,164,TRANGNNT2_VTPOST_HCM,TRANGNNT2_VTPOST_HCM,3000000.0,10000.0,2,400300,5500.0,1800590021,11000.0,vu thi thanh giang,LE TUNG LAM,0001c4bc0350dc5c8c80,0001c4bc0350dc5c8c80,000145eb7b76f4e77de5,Nguyễn Ngọc Thùy Trang
2,D061,D061003,VTT,6,2200190007,DAUNT2_DNI_DLXNK,600000.0,0.0,2,400100,1260.0,2200190007,0.0,NGUYỄN VĂN TIỀN,NGUYỄN VĂN TIỀN,00011341b2c0b40071f7,00011341b2c0b40071f7,0001d5dcf92a0dab9da1,Nguyễn Tranh Đấu
3,T008,T00801,KPP,197,6600240001,6600240001,393692.0,1000.0,3,PAY_BILL,770.0,6600240001,0.0,None,None,None,None,0001c1333ab419853e05,None
4,C026,C026001,KPP,1,1000113100_00004_DBCN,1000113100_00004_DBCN,349384.0,909.0,3,KPP301,700.0,1000113100_VTP,0.0,None,None,00019784c19ed12c546c,None,00019784c19ed12c546c,Diem CCDVVT LD_Dang Dinh Lam


In [9]:
train_df.shape

(7193506, 19)

In [10]:
train_df = train_df.merge(hist_df, how='left', left_on=['ben_msisdn', 'staff_name'], right_on=['msisdn_', 'ben_cust_name_'])
train_df.head()

,province_code,district_code,viettel_bank_code,channel_type_id,channel_code,staff_code,trans_amount,trans_fee,trans_type,process_code,channel_fee,shop_code,customer_fee,cust_name,ben_cust_name,msisdn,ben_msisdn,msisdn_channel,staff_name,msisdn_,ben_cust_name_,trans_amount_sum,trans_amount_mean
0,D501,D501008,MB,164,HOANT35_VTPOST_DKN,HOANT35_VTPOST_DKN,500000.0,8182.0,1,645200,5400.0,1800170007,18000.0,HỒ THỊ KIM TRANG,MAI VAN QUY,0001b3807ce1879a06eb,0001b3807ce1879a06eb,000106f0196c5e4f0853,Nguyễn Thị Hoa,NaN,NaN,NaN,NaN
1,T008,T00807,MB,164,TRANGNNT2_VTPOST_HCM,TRANGNNT2_VTPOST_HCM,3000000.0,10000.0,2,400300,5500.0,1800590021,11000.0,vu thi thanh giang,LE TUNG LAM,0001c4bc0350dc5c8c80,0001c4bc0350dc5c8c80,000145eb7b76f4e77de5,Nguyễn Ngọc Thùy Trang,NaN,NaN,NaN,NaN
2,D061,D061003,VTT,6,2200190007,DAUNT2_DNI_DLXNK,600000.0,0.0,2,400100,1260.0,2200190007,0.0,NGUYỄN VĂN TIỀN,NGUYỄN VĂN TIỀN,00011341b2c0b40071f7,00011341b2c0b40071f7,0001d5dcf92a0dab9da1,Nguyễn Tranh Đấu,NaN,NaN,NaN,NaN
3,T008,T00801,KPP,197,6600240001,6600240001,393692.0,1000.0,3,PAY_BILL,770.0,6600240001,0.0,None,None,None,None,0001c1333ab419853e05,None,NaN,NaN,NaN,NaN
4,C026,C026001,KPP,1,1000113100_00004_DBCN,1000113100_00004_DBCN,349384.0,909.0,3,KPP301,700.0,1000113100_VTP,0.0,None,None,00019784c19ed12c546c,None,00019784c19ed12c546c,Diem CCDVVT LD_Dang Dinh Lam,NaN,NaN,NaN,NaN


In [14]:
train_df.dtypes.value_counts()

object     17
float64     6
dtype: int64

In [13]:
num_cols = train_df._get_numeric_data().columns
cat_cols = list(set(train_df.columns) - set(num_cols))
cat_cols

['viettel_bank_code',
 'msisdn_channel',
 'msisdn_',
 'msisdn',
 'district_code',
 'ben_cust_name_',
 'process_code',
 'staff_code',
 'ben_msisdn',
 'shop_code',
 'province_code',
 'channel_code',
 'cust_name',
 'trans_type',
 'staff_name',
 'ben_cust_name',
 'channel_type_id']

In [ ]:
cat_features = ['viettel_bank_code',
 'msisdn_channel',
 'msisdn_',
 'msisdn',
 'district_code',
 'ben_cust_name_',
 'process_code',
 'staff_code',
 'ben_msisdn',
 'shop_code',
 'province_code',
 'channel_code',
 'cust_name',
 'trans_type',
 'staff_name',
 'ben_cust_name',
 'channel_type_id']


In [ ]:
train['request_date_dt'] = pd.to_datetime(train['request_date'], format='%Y-%m-%d %H:%M:%S')
train['date_diff'] = (train['request_date_dt'] - train['request_date_dt'].min()).dt.days
train.head()

In [ ]:
train.shape

In [ ]:
train['is_fraud'].value_counts().sort_values()

In [ ]:
train[['is_fraud' > 1]].count()

In [ ]:
train['is_fraud'].astype(int).plot.hist()

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# Missing values statistics
missing_values = missing_values_table(train)

In [ ]:
missing_values.head(200)

In [ ]:
data.dtypes.value_counts()

In [ ]:
data.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

In [ ]:
# Number of NaNs for each object
data.isnull().sum(axis=1).head(30)

In [ ]:
# Number of NaNs for each column
data.isnull().sum(axis=0).head(30)

In [ ]:
#Remove constant features
# `dropna = False` makes nunique treat NaNs as a distinct value
feats_counts = data.nunique(dropna = False)

In [ ]:
feats_counts.sort_values()[:10]

In [ ]:
# constant_features = feats_counts.loc[feats_counts==1].index.tolist()
# print (constant_features)

# data.drop(constant_features,axis = 1,inplace=True)

In [ ]:
# Remove duplicated features
data.fillna('NaN', inplace=True)
train_enc =  pd.DataFrame(index = data.index)

for col in tqdm_notebook(data.columns):
    train_enc[col] = data[col].factorize()[0]
    
dup_cols = {}

for i, c1 in enumerate(tqdm_notebook(train_enc.columns)):
    for c2 in train_enc.columns[i + 1:]:
        if c2 not in dup_cols and np.all(train_enc[c1] == train_enc[c2]):
            dup_cols[c2] = c1
dup_cols

In [ ]:
# examine the number of unique values.
nunique = data.nunique(dropna=False)
nunique

In [ ]:
plt.figure(figsize=(14,6))
_ = plt.hist(nunique.astype(float)/data.shape[0], bins=100)

In [ ]:
mask = (nunique.astype(float)/data.shape[0] > 0.25)
data.loc[:, mask]

In [ ]:
mask = (nunique.astype(float)/data.shape[0] < 0.4) & (nunique.astype(float)/data.shape[0] > 0.2)
data.loc[:25, mask]

In [ ]:
cat_cols = list(data.select_dtypes(include=['object']).columns)
num_cols = list(data.select_dtypes(exclude=['object']).columns)

In [ ]:
data.replace('NaN', -999, inplace=True)

In [ ]:
def autolabel(arrayA):
    ''' label each colored square with the corresponding data value. 
    If value > 20, the text is in black, else in white.
    '''
    arrayA = np.array(arrayA)
    for i in range(arrayA.shape[0]):
        for j in range(arrayA.shape[1]):
                plt.text(j,i, "%.2f"%arrayA[i,j], ha='center', va='bottom',color='w')

def hist_it(feat):
    plt.figure(figsize=(16,4))
    feat[Y==0].hist(bins=range(int(feat.min()),int(feat.max()+2)),normed=True,alpha=0.8)
    feat[Y==1].hist(bins=range(int(feat.min()),int(feat.max()+2)),normed=True,alpha=0.5)
    plt.ylim((0,1))
    
def gt_matrix(feats,sz=16):
    a = []
    for i,c1 in enumerate(feats):
        b = [] 
        for j,c2 in enumerate(feats):
            mask = (~data[c1].isnull()) & (~data[c2].isnull())
            if i>=j:
                b.append((data.loc[mask,c1].values>=data.loc[mask,c2].values).mean())
            else:
                b.append((data.loc[mask,c1].values>data.loc[mask,c2].values).mean())

        a.append(b)

    plt.figure(figsize = (sz,sz))
    plt.imshow(a, interpolation = 'None')
    _ = plt.xticks(range(len(feats)),feats,rotation = 90)
    _ = plt.yticks(range(len(feats)),feats,rotation = 0)
    autolabel(a)

In [ ]:
# # select first 42 numeric features
# feats = num_cols[:42]

# # build 'mean(feat1 > feat2)' plot
# gt_matrix(feats,16)

In [ ]:
data.corr()

In [ ]:
plt.matshow(data.corr())
plt.show()

In [ ]:
sns.heatmap(data.corr(), annot=True)
plt.show()